A cute little demo showing the simplest usage of minGPT. Configured to run fine on Macbook Air in like a minute.

In [1]:
import torch
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
from mingpt.utils import set_seed
from mingpt.bpe import BPETokenizer
set_seed(3407)

In [2]:
import pickle

class SortDataset(Dataset):
    """ 
    Dataset for the Sort problem. E.g. for problem length 6:
    Input: 0 0 2 1 0 1 -> Output: 0 0 0 1 1 2
    Which will feed into the transformer concatenated as:
    input:  0 0 2 1 0 1 0 0 0 1 1
    output: I I I I I 0 0 0 1 1 2
    where I is "ignore", as the transformer is reading the input sequence
    """

    def __init__(self, split, length=6, num_digits=3):
        assert split in {'train', 'test'}
        self.split = split
        self.length = length
        self.num_digits = num_digits
    
    def __len__(self):
        return 10000 # ...
    
    def get_vocab_size(self):
        return self.num_digits
    
    def get_block_size(self):
        # the length of the sequence that will feed into transformer, 
        # containing concatenated input and the output, but -1 because
        # the transformer starts making predictions at the last input element
        return self.length * 2 - 1

    def __getitem__(self, idx):
        
        # use rejection sampling to generate an input example from the desired split
        while True:
            # generate some random integers
            inp = torch.randint(self.num_digits, size=(self.length,), dtype=torch.long)
            # half of the time let's try to boost the number of examples that 
            # have a large number of repeats, as this is what the model seems to struggle
            # with later in training, and they are kind of rate
            if torch.rand(1).item() < 0.5:
                if inp.unique().nelement() > self.length // 2:
                    # too many unqiue digits, re-sample
                    continue
            # figure out if this generated example is train or test based on its hash
            h = hash(pickle.dumps(inp.tolist()))
            inp_split = 'test' if h % 4 == 0 else 'train' # designate 25% of examples as test
            if inp_split == self.split:
                break # ok
        
        # solve the task: i.e. sort
        sol = torch.sort(inp)[0]

        # concatenate the problem specification and the solution
        cat = torch.cat((inp, sol), dim=0)

        # the inputs to the transformer will be the offset sequence
        x = cat[:-1].clone()
        y = cat[1:].clone()
        # we only want to predict at output locations, mask out the loss at the input locations
        y[:self.length-1] = -1
        return x, y

In [11]:
class TweetLMDataset(torch.utils.data.Dataset):
    def __init__(self, df, max_length=128, min_length=1):
        self.df = df
        self.tokenizer = BPETokenizer()
        self.min_length = min_length
        self.max_length = max_length
    
    def get_vocab_size(self):
        return self.tokenizer.get_vocab_size()
    
    def get_block_size(self):
        return self.max_length

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        # Get the tweet text
        tweet = self.df.iloc[idx]['text']
        
        # Randomly choose a length between min_length and max_length using torch
        target_length = torch.randint(self.min_length, len(tweet)+1, (1,)).item()
        tweet = tweet[:target_length]
        
        # Tokenize the tweet with padding and attention mask
        input_ids = self.tokenizer(tweet).flatten()
        
        # For next-token prediction:
        # input_ids: [token1, token2, token3, ..., tokenN]
        # labels:    [token2, token3, token4, ..., -1]
        # The model should predict each token given all previous tokens
        
        # Create labels by shifting input_ids right by 1
        # This means each position in labels is the token that should be predicted
        # given all tokens before that position in input_ids
        labels = input_ids.clone()
        labels[:-1] = input_ids[1:]  # Shift right by 1
        labels[-1] = -1  # Last position should be ignored as we don't have the next token
        
        # attention mask
        attention_mask = torch.ones((target_length,), dtype=torch.int32)

        # pad to max length
        pad_length = self.max_length - target_length
        input_ids = torch.nn.functional.pad(input_ids, (0, pad_length))
        labels = torch.nn.functional.pad(labels, (0, pad_length), valuue=-1)
        attention_mask = torch.nn.functional.pad(attention_mask, (0, pad_length))


        return input_ids, labels, attention_mask

In [12]:
# print an example instance of the dataset
train_dataset = SortDataset('train')
test_dataset = SortDataset('test')
x, y = train_dataset[0]
print(x, y)

tensor([1, 0, 1, 0, 0, 2, 0, 0, 0, 1, 1]) tensor([-1, -1, -1, -1, -1,  0,  0,  0,  1,  1,  2])


In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('data/Tweets.csv')
df_train, df_test = train_test_split(df, test_size=0.1, random_state=42)

train_dataset = TweetLMDataset(df_train)
test_dataset = TweetLMDataset(df_test)

In [17]:
x, y, attn = train_dataset[0]
print(x.shape, y.shape, attn.shape)

torch.Size([83]) torch.Size([83]) torch.Size([128])


In [15]:
# create a GPT instance
from mingpt.model import GPT

model_config = GPT.get_default_config()
model_config.model_type = 'gpt-nano'
model_config.vocab_size = train_dataset.get_vocab_size()
model_config.block_size = train_dataset.get_block_size()
model = GPT(model_config)

number of parameters: 2.50M


In [16]:
# create a Trainer object
from mingpt.trainer import Trainer

train_config = Trainer.get_default_config()
train_config.learning_rate = 5e-4 # the model we're using is so small that we can go a bit faster
train_config.max_iters = 2000
train_config.num_workers = 0
train_config.epochs = 10
trainer = Trainer(train_config, model, train_dataset)

running on device cpu


In [9]:
def batch_end_callback(trainer):
    if trainer.iter_num % 100 == 0:
        print(f"iter_dt {trainer.iter_dt * 1000:.2f}ms; iter {trainer.iter_num}: train loss {trainer.loss.item():.5f}")
trainer.set_callback('on_batch_end', batch_end_callback)

trainer.run()

/Users/advaitkoparkar/Documents/projects/minRLHF/.minrlhf_env/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


RuntimeError: stack expects each tensor to be equal size, but got [110] at entry 0 and [107] at entry 1

In [14]:
# now let's perform some evaluation
model.eval();

In [15]:
def eval_split(trainer, split, max_batches):
    dataset = {'train':train_dataset, 'test':test_dataset}[split]
    n = train_dataset.length # naugy direct access shrug
    results = []
    mistakes_printed_already = 0
    loader = DataLoader(dataset, batch_size=100, num_workers=0, drop_last=False)
    for b, (x, y) in enumerate(loader):
        x = x.to(trainer.device)
        y = y.to(trainer.device)
        # isolate the input pattern alone
        inp = x[:, :n]
        sol = y[:, -n:]
        # let the model sample the rest of the sequence
        cat = model.generate(inp, n, do_sample=False) # using greedy argmax, not sampling
        sol_candidate = cat[:, n:] # isolate the filled in sequence
        # compare the predicted sequence to the true sequence
        correct = (sol == sol_candidate).all(1).cpu() # Software 1.0 vs. Software 2.0 fight RIGHT on this line haha
        for i in range(x.size(0)):
            results.append(int(correct[i]))
            if not correct[i] and mistakes_printed_already < 3: # only print up to 5 mistakes to get a sense
                mistakes_printed_already += 1
                print("GPT claims that %s sorted is %s but gt is %s" % (inp[i].tolist(), sol_candidate[i].tolist(), sol[i].tolist()))
        if max_batches is not None and b+1 >= max_batches:
            break
    rt = torch.tensor(results, dtype=torch.float)
    print("%s final score: %d/%d = %.2f%% correct" % (split, rt.sum(), len(results), 100*rt.mean()))
    return rt.sum()

# run a lot of examples from both train and test through the model and verify the output correctness
with torch.no_grad():
    train_score = eval_split(trainer, 'train', max_batches=50)
    test_score  = eval_split(trainer, 'test',  max_batches=50)

train final score: 5000/5000 = 100.00% correct
test final score: 5000/5000 = 100.00% correct


In [16]:
# let's run a random given sequence through the model as well
n = train_dataset.length # naugy direct access shrug
inp = torch.tensor([[0, 0, 2, 1, 0, 1]], dtype=torch.long).to(trainer.device)
assert inp[0].nelement() == n
with torch.no_grad():
    cat = model.generate(inp, n, do_sample=False)
sol = torch.sort(inp[0])[0]
sol_candidate = cat[:, n:]
print('input sequence  :', inp.tolist())
print('predicted sorted:', sol_candidate.tolist())
print('gt sort         :', sol.tolist())
print('matches         :', bool((sol == sol_candidate).all()))

input sequence  : [[0, 0, 2, 1, 0, 1]]
predicted sorted: [[0, 0, 0, 1, 1, 2]]
gt sort         : [0, 0, 0, 1, 1, 2]
matches         : True
